## Import the dependencies

In [1]:
import pandas as pd #for data manipulation
from sklearn.model_selection import GridSearchCV #for hyperparamwter tuning of the machine learning model
from sklearn.ensemble import RandomForestClassifier #for building a classification model
from sklearn.preprocessing import OneHotEncoder #for encoding categorical features
from sklearn.impute import SimpleImputer #for imputing missing values
from sklearn.compose import make_column_selector #for selecting specific columns from a dataframe
from sklearn.compose import make_column_transformer #for creating a transformer for different columns in a dataframe
from sklearn.pipeline import make_pipeline #for chaining multiple preprocessing steps and a machine learning model
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler #for scaling numerical features
from sklearn.feature_extraction.text import TfidfVectorizer #for vectorizing text data
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 

## Data collection and analysis

In [2]:
# Read in the Titanic dataset
titanic=pd.read_csv("titanic.csv")

# Print the shape and first 15 rows of the dataset
print(titanic.shape)
titanic.head(15)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
#Getting some information about the data
titanic.info()

#Check the number of missing values in each column
titanic.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
#Feature selection: Remove the "PassengerId" and "Cabin" columns, shuffle the rows and reassign the dataframe to titanic
titanic=titanic.drop(columns=["PassengerId","Cabin"]).sample(frac=1,random_state=55)
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
713,0,3,"Larsson, Mr. August Viktor",male,29.0,0,0,7545,9.4833,S
726,1,2,"Renouf, Mrs. Peter Henry (Lillian Jefferys)",female,30.0,3,0,31027,21.0000,S
262,0,1,"Taussig, Mr. Emil",male,52.0,1,1,110413,79.6500,S
266,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,3101295,39.6875,S
186,1,3,"O'Brien, Mrs. Thomas (Johanna ""Hannah"" Godfrey)",female,NaN,1,0,370365,15.5000,Q


In [5]:
titanic.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

## Separate features and target

In [6]:
# Separate the target variable (y) from the rest of the features (X)
y=titanic.pop("Survived")
X=titanic

In [7]:
print(X.shape)
print(y.shape)

(891, 9)
(891,)


## Preprocessing

In [8]:
# Define the preprocessing steps in a Preprocessing class
class Preprocessing:
    # Define a one-hot encoder, median imputer, constant imputer, TF-IDF vectorizer, and standard scaler
    ohe=OneHotEncoder()
    imp_median=SimpleImputer(strategy="median",add_indicator=True)
    imp_constant=SimpleImputer(strategy="constant")
    tfidf=TfidfVectorizer()
    scaler=StandardScaler()
    # Define a column transformer that applies the preprocessing steps to the appropriate columns
    ct=make_column_transformer(
        (make_pipeline(imp_median,scaler),make_column_selector(dtype_include="number")),
        (make_pipeline(imp_constant,ohe),["Sex","Embarked"]),
        (tfidf,"Name"))

In [9]:
#Create a class that holds the hyperparameter grids for each model
class ParamGrid:
    # Grid for Random Forest model
    rf_param_grid = {"clf__n_estimators": [100, 200, 300],
                         "clf__max_depth": [None, 10, 20]};
    #Grid for Logistic Regrression model
    logreg_param_grid = {"clf__penalty": ["l1", "l2"],
                          "clf__C": [0.1, 1, 10],
                          "clf__solver": ["liblinear", "saga"]};
    #Grid for SVM model
    svm_param_grid = {"clf__C": [0.1, 1, 10],
                          "clf__kernel": ["linear", "rbf","poly"],
                          "clf__gamma": ["scale", "auto"]};

## Model training and evaluation

In [10]:
#Create a class that represents a model
class TitanicModel:
    def __init__(self):
        # Create a list of tuples where each tuple contains the name of the model and an instance of the model class
        self.models=[("logreg", LogisticRegression(max_iter=250)), ("rf", RandomForestClassifier()), ("svm", SVC())]
        # Create a dictionary to hold the best model and its hyperparameters
        self.best_model = {"name": None, "model": None, "params": None, "score": 0}

    def fit(self):
        for name, model in self.models:
            # Create a Pipeline object that contains the necessary preprocessing steps and the model
            pipeline = Pipeline([("preprocessor",Preprocessing.ct),("clf", model)])

            # Define the parameter grid for the model
            if name == "logreg":
                param_grid = ParamGrid.logreg_param_grid
            elif name == "rf":
                param_grid = ParamGrid.rf_param_grid
            elif name == "svm":
                param_grid = ParamGrid.svm_param_grid

            # Perform a grid search over the hyperparameters for the model
            search = GridSearchCV(pipeline, param_grid=param_grid, scoring="accuracy", cv=5,n_jobs=-1)
            search.fit(X,y)

            # Update the dictionary with the best model and its hyperparameters if it has a better score than the current best model
            if search.best_score_> self.best_model["score"]:
                self.best_model["name"] = name
                self.best_model["model"] = search.best_estimator_
                self.best_model["params"] = search.best_params_
                self.best_model["score"] = search.best_score_
    #Print the name,best estimator,best parameters and best score from the best_model dctionary          
    def result(self):
        print(self.best_model)

In [11]:
#Create a TitanicModel object
model=TitanicModel()
#Fit the model with the data
model.fit()
#Get the result from the grid search
model.result()

{'name': 'rf', 'model': Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021FC8E4D880>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='constant')),
                           